In [50]:
from collections import Counter
import math
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy
import time
import itertools

In [7]:
ratings = pd.read_csv('ml-latest-small/ratings_train.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [58]:
movies.head(3001)

0       [Adventure, Animation, Children, Comedy, Fantasy]
1                          [Adventure, Children, Fantasy]
2                                       [Comedy, Romance]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
2996                            [Comedy, Crime, Thriller]
2997                                      [Comedy, Drama]
2998                                     [Drama, Romance]
2999                                     [Comedy, Sci-Fi]
3000    [Adventure, Animation, Children, Comedy, Fantasy]
Name: tokens, Length: 3001, dtype: object

In [9]:
def tokenize(movies):

#   add a new column to the movies, which contains lists of words from genres
    tokenlist=[]
    for index,row in movies.iterrows():
        tokenlist.append(tokenize_string(row.genres))
    movies['tokens']=tokenlist
    return movies

def tokenize_string(my_string):
#     split by |
    return my_string.split('|');

In [10]:
movies_tokenized = tokenize(movies)
print(movies_tokenized.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  \
0  Adventure|Animation|Children|Comedy|Fantasy   
1                   Adventure|Children|Fantasy   
2                               Comedy|Romance   
3                         Comedy|Drama|Romance   
4                                       Comedy   

                                              tokens  
0  [Adventure, Animation, Children, Comedy, Fantasy]  
1                     [Adventure, Children, Fantasy]  
2                                  [Comedy, Romance]  
3                           [Comedy, Drama, Romance]  
4                                           [Comedy]  


In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
mlb = MultiLabelBinarizer()
df = movies_tokenized.join(pd.DataFrame(mlb.fit_transform(movies_tokenized['tokens']),columns=mlb.classes_))

In [13]:
movies = movies_tokenized['tokens']

In [14]:
df = df.drop(['title', 'genres', 'tokens', 'features'], axis=1)

KeyError: "['features'] not found in axis"

In [ ]:
df = df.to_numpy()

In [15]:
matr = df.T

In [16]:
matr = matr[1:]

In [17]:
matr

,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movi...,Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
genres,Adventure|Animation|Children|Comedy|Fantasy,Adventure|Children|Fantasy,Comedy|Romance,Comedy|Drama|Romance,Comedy,Action|Crime|Thriller,Comedy|Romance,Adventure|Children,Action,Action|Adventure|Thriller,...,Action|Animation|Comedy|Sci-Fi,Animation|Drama,Comedy|Drama,Animation,Documentary,Action|Animation|Comedy|Fantasy,Animation|Comedy|Fantasy,Drama,Action|Animation,Comedy
tokens,"[Adventure, Animation, Children, Comedy, Fantasy]","[Adventure, Children, Fantasy]","[Comedy, Romance]","[Comedy, Drama, Romance]",[Comedy],"[Action, Crime, Thriller]","[Comedy, Romance]","[Adventure, Children]",[Action],"[Action, Adventure, Thriller]",...,"[Action, Animation, Comedy, Sci-Fi]","[Animation, Drama]","[Comedy, Drama]",[Animation],[Documentary],"[Action, Animation, Comedy, Fantasy]","[Animation, Comedy, Fantasy]",[Drama],"[Action, Animation]",[Comedy]
(no genres listed),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Action,0,0,0,0,0,1,0,0,1,1,...,1,0,0,0,0,1,0,0,1,0
Adventure,1,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
Animation,1,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,1,1,0,1,0
Children,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Comedy,1,0,1,1,1,0,1,0,0,0,...,1,0,1,0,0,1,1,0,0,1
Crime,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def tf(word, doc):
    # tf(i, d) / max_k tf(k, d)
    return doc.count(word) / Counter(doc).most_common()[0][1]

def df(word, doclist):
    # df(i)
    return sum(1 for d in doclist if word in d)

def tfidf(word, doc, dfdict, N):
    return tf(word, doc) * math.log2((N / dfdict[word]))

def getcsrmatrix(tokens, dfdict, N, vocab):
    matrixRow_list = []
    matrixRow_list = np.zeros((1, len(vocab)), dtype='float')
    for t in tokens:
        if t in vocab:
            # use number to replace the word then add the score
            matrixRow_list[0][vocab[t]] = tfidf(t, tokens, dfdict, N)
    # output a sparse one
    return csr_matrix(matrixRow_list)

def add_tfidf_feature (movies_tokenized):
    """
    Returns:
      A tuple containing:
      - The movies DataFrame, which has been modified to include a column named 'features'.
      - The vocab, a dict from term to int. Make sure the vocab is sorted alphabetically as in a2 (e.g., {'a': 0, 'b': 1, ...})
    """

#   doclist example:   ['adventure', 'animation', 'children', 'comedy', 'fantasy'], ['adventure', 'children', 'fantasy'], 
#     ['comedy', 'romance'], ['comedy', 'drama', 'romance'], ['comedy']
    doclist = movies_tokenized['tokens'].tolist() 
    
    vocab_set = set(i for s in doclist for i in s)
    vocab = {i: x for x, i in enumerate(sorted(list(vocab_set)))}

# [('(no genres listed)', 0), ('Action', 1), ('Adventure', 2), ('Animation', 3), ('Children', 4), ('Comedy', 5), ('Crime', 6), ('Documentary', 7), ('Drama', 8), ('Fantasy', 9),..... ]  
    
    dfdict = {}
    
#     {'Action': 1828, 'Adventure': 1263, 'Animation': 611, 'Children': 664, 'Comedy': 3756, 'Crime': 1199, 'Documentary': 440, 'Drama': 4361, 'Fantasy': 779,....}
    for v in vocab.items():
        dfdict[v[0]] = df(v[0], doclist)

    print(dfdict)
    csrlist = []

    N = len(movies_tokenized)
    
    #     dataframe.iterrows()
    for index, row in movies_tokenized.iterrows():
        csrlist.append(getcsrmatrix(row['tokens'], dfdict, N, vocab))
    print(csrlist[0])
    movies_tokenized['features'] = csrlist
    return (movies_tokenized, vocab)


In [9]:
movies_features, vocab = add_tfidf_feature(movies_tokenized)

{'(no genres listed)': 34, 'Action': 1828, 'Adventure': 1263, 'Animation': 611, 'Children': 664, 'Comedy': 3756, 'Crime': 1199, 'Documentary': 440, 'Drama': 4361, 'Fantasy': 779, 'Film-Noir': 87, 'Horror': 978, 'IMAX': 158, 'Musical': 334, 'Mystery': 573, 'Romance': 1596, 'Sci-Fi': 980, 'Thriller': 1894, 'War': 382, 'Western': 167}
  (0, 2)	2.947363344052896
  (0, 3)	3.9949736980217185
  (0, 4)	3.8749628364935234
  (0, 5)	1.3750209201866737
  (0, 9)	3.644522749778779


In [10]:
vocab

{'(no genres listed)': 0,
 'Action': 1,
 'Adventure': 2,
 'Animation': 3,
 'Children': 4,
 'Comedy': 5,
 'Crime': 6,
 'Documentary': 7,
 'Drama': 8,
 'Fantasy': 9,
 'Film-Noir': 10,
 'Horror': 11,
 'IMAX': 12,
 'Musical': 13,
 'Mystery': 14,
 'Romance': 15,
 'Sci-Fi': 16,
 'Thriller': 17,
 'War': 18,
 'Western': 19}

In [8]:
def train_test_split(ratings):

#     Returns a random split of the ratings matrix into a training and testing set.
    test = set(range(len(ratings))[::1000])
    train = sorted(set(range(len(ratings))) - test)
    test = sorted(test)
    return ratings.iloc[train], ratings.iloc[test]

In [9]:
ratings_train, ratings_test = train_test_split(ratings)

In [10]:
print('{} training ratings; {} testing ratings' .format (len(ratings_train), len(ratings_test)))

90661 training ratings; 91 testing ratings


In [18]:
print(ratings_test.head())

      userId  movieId  rating   timestamp
0          1        3     4.0   964981247
1000       9     5952     5.0  1044656908
2000      18   112421     4.0  1456832692
3000      21     3623     4.0  1376823417
4000      28     4238     2.5  1242290762


In [13]:
def cosine_sim(a, b):
    """
    Params:
      a...A csr_matrix with shape (1, number_features)
      b...A csr_matrix with shape (1, number_features)
    Returns:
      The cosine similarity, defined as: dot(a, b) / ||a|| * ||b||
      where ||a|| indicates the Euclidean norm (aka L2 norm) of vector a.
    """
    # here turn the sparse matrix to the array which may contain many zeros, and then compute the cosine similarity
    v1 = a.toarray()[0]
    v2  = b.toarray()[0]

    return sum(i[0] * i[1] for i in zip(v1, v2))/(math.sqrt(sum([i*i for i in v1]))*math.sqrt(sum([i*i for i in v2])))

def rating_calcu_cossim (movies_features, ratings_train, ratings_test):  
    """
     Return a numpy array containing one recommended rating for each element of ratings_test.
    """
    result = []
    
#   index -> row_number, row -> the four columns value of that line
    for index, row in ratings_test.iterrows():
        
        # find the movieId that train user_id equals the test user_id 
        mlist = list(ratings_train.loc[ratings_train['userId'] == row['userId']]['movieId'])
    
        # find the features the movie have if the movies are in the mlist
        csrlist = list(movies_features.loc[movies_features['movieId'].isin(mlist)]['features'])

        # movies rating by that user
        mrlist = list(ratings_train.loc[ratings_train['userId'] == row['userId']]['rating'])
    
        # cosine similarity calculation
        cmlist = [cosine_sim(c, movies_features.loc[movies_features['movieId'] == row['movieId']]['features'].values[0]) for c in csrlist]
            
        # sum_ratings is the sum of the ratings if the cosine sim > 0, multiply the ratings with the cos_sim 
        sum_ratings = sum([v * mrlist[i] for i, v in enumerate(cmlist) if v > 0])
        
        # pos_sim is the list that cosine sim > 0
        pos_sim = [i for i in cmlist if i > 0]
        
        # if have positive cos_sim
        if (len(pos_sim) > 0):
            result.append(sum_ratings / sum(pos_sim))
        # if there is no positive cos_sim
        else:
            result.append(np.mean(mrlist))
    return np.array(result)


In [14]:
recommend_cos = rating_calcu_cossim(movies_features, ratings_train, ratings_test)

In [19]:
recommend_cos

array([4.29048953, 3.85365151, 3.84361452, 3.55091839, 2.96067213,
       3.90851143, 3.87059832, 3.83114853, 4.15064712, 4.00681414,
       3.25870895, 3.33331876, 3.83611647, 3.62543669, 3.22955858,
       4.15371582, 3.42508938, 4.53778213, 3.19890312, 3.44816375,
       2.7968865 , 2.84926067, 4.06361123, 3.24056979, 3.78074715,
       3.84303264, 3.37329212, 3.45921016, 2.88828894, 4.0022022 ,
       2.78529069, 3.4956384 , 3.980301  , 3.7679177 , 3.81955437,
       4.1145389 , 3.33559659, 3.66037931, 3.24802462, 4.19844495,
       4.69890808, 2.88616065, 3.50171756, 3.38005633, 3.91121315,
       3.56149025, 3.82059465, 4.14191463, 3.77005227, 3.97493683,
       2.70535185, 3.90121407, 3.63485936, 4.00203087, 3.0136137 ,
       4.5407045 , 4.06299791, 3.52289531, 3.24623472, 3.68528667,
       4.05275108, 3.33608038, 2.68801532, 2.80119537, 4.00733459,
       3.98190966, 3.56548656, 3.40396371, 3.73469116, 3.19092981,
       3.64226435, 3.43779371, 3.51954167, 3.44487396, 3.91647

In [15]:
def mean_absolute_error(recommend_cos, ratings_test):
#     Return the mean absolute error of the recommend ratings.
    return np.abs(recommend_cos - np.array(ratings_test.rating)).mean()

In [16]:
print('error= {}'.format(mean_absolute_error(recommend_cos, ratings_test)))

error= 0.6969762311313223


In [18]:
def get_hash_config(k_shingle):
    rnds = np.random.choice(2**10, (2, k_shingle), replace=False)
    c = 1048583
    return rnds[0], rnds[1], c

In [40]:
def min_hashing(matrix, hash_config, k_shingle):
    count = matrix.shape[1]
   
    (a, b, c) = hash_config
    a = a.reshape(1, -1)
    M = np.zeros((k_shingle, count), dtype=int)
    for i in range(count):
        row_idx = matrix[:, i].indices.reshape(-1, 1)
      
        m = (np.matmul(row_idx, a) + b) % c
        if len(m)>0:
            m_min = np.min(m, axis=0)
            M[:, i] = m_min

    return M


In [53]:
def LSH(M, b, r, k_band):
    lines = M.shape[1]

    bucket_list = []
    for band_index in range(b):
        row_idx = []
        col_idx = []

        row_start = band_index * r
        for c in range(lines):
            v = M[row_start:(row_start+r), c]
            v_hash = hash(tuple(v.tolist())) % k_band
            row_idx.append(v_hash)
            col_idx.append(c)
            #m[v_hash, c] = 1
        data_ary = [True] * len(row_idx)

        m = scipy.sparse.csr_matrix((data_ary, (row_idx, col_idx)), shape=(k_band, lines), dtype=bool)
        bucket_list.append(m)
  
    return bucket_list


In [54]:
def find_all_similiar(bucket_list, M, b, k_band, verify_by_large):
    e1 = time.time()
    candidates = set() # set of frozenset
    for band in bucket_list:
        for i in range(k_band):
            bucket = band[i].indices
            if len(bucket) >= 2:
                pairs = itertools.combinations(bucket, 2)
                pair_sets = [frozenset(p) for p in pairs]
                candidates = candidates.union(pair_sets)
    e2 = time.time()
    print('Done first pass', e2-e1)
    print('#C', len(candidates))

    #pdb.set_trace()
    #matrix = scipy.sparse.csc_matrix(matrix)
    # step4, filtering based on hash
    candidate_list = []
    if verify_by_large:
        for pair in candidates:
            i1, i2 = pair
            c1 = set(matrix[:, i1].indices)
            c2 = set(matrix[:, i2].indices)
            sim = len(c1 & c2) / len(c1 | c2)
            if sim >= 0.8:
                candidate_list.append((pair, sim))
    else:
        for pair in candidates:
            i1, i2 = pair
            c1 = M[:, i1]
            c2 = M[:, i2]
            sim = np.mean(c1 == c2)
            if sim >= 0.8:
                candidate_list.append((pair, sim))

    e3 = time.time()
    candidate_list = sorted(candidate_list, key=lambda x: x[1], reverse=True)
    print('Done 2nd pass', e3-e2)
    return candidate_list

In [55]:
filename = 'ml-latest-small/movies.csv'

n_grams = 5
hash_size = 2**16

b = 10
r = 10
k_shingle = b * r
k_band = 2**16
verify_by_large = False

matrix = shingle_hash(n_grams, hash_size, filename)

hash_config = get_hash_config(k_shingle)


M = min_hashing(matrix, hash_config, k_shingle)
bucket_list = LSH(M, b, r, k_band)

candidates_list2 = find_all_similiar(bucket_list, M, b, k_band, verify_by_large)

print('All of similiar pairs')
print(candidates_list2)
print(len(candidates_list2))

Done first pass 643.1033461093903
#C 1445615
Done 2nd pass 24.458448886871338
All of similiar pairs


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
matrix = shingle_hash(n_grams, hash_size, filename)
M = min_hashing(matrix, hash_config, k_shingle)

In [32]:
# def find_ngrams(input_list, n):
#     return zip(*[input_list[i:] for i in range(n)])
def shingle_hash(n_grams, hash_size, filename):

    row_idx = []
    col_idx = []
    lines = 0
    df = pd.read_csv(filename)
    for i, row in df.iterrows():
#     for i, line in enumerate(open(filename, 'r', encoding='utf-8')):
        lines += 1
#         tokens = [t for t in line[:-1].split(',') if  t != '']
       
        tokens = row['genres'].split('|')
        
#         hashed = []
#         for tok in tokens:
#             hashed.append(vocab[tok])
        ngram = frozenset(tokens)
#         print(ngram)
        _hash = hash(ngram) % hash_size
#         print(_hash)
        row_idx.append(_hash)
        col_idx.append(i)

    
    # I did a few benchmark, although lil_matrix, dok_matrix are claimed to be efficient, they are not as efficient as array
    # So it's faster to record non-zero
    data_ary = [True] * len(row_idx)

    # Convert to column based matrix for fast processing
   
    to_return = scipy.sparse.csc_matrix((data_ary, (row_idx, col_idx)), shape=(hash_size, lines), dtype=bool)
  
    return to_return